In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Tugas: Eksplorasi dan Reproduksi Model Deteksi Penyakit Daun Tomat\n",
    "\n",
    "**Tujuan Notebook:**\n",
    "1.  **Reproduksi:** Mereplikasi hasil dari notebook Kaggle referensi ([Tomato Leaf Disease (94% accuracy)](https://www.kaggle.com/code/samanfatima7/tomato-leaf-disease-94-accuracy)) yang menggunakan Custom CNN.\n",
    "2.  **Eksplorasi:** Mencoba meningkatkan performa model menggunakan teknik Transfer Learning (EfficientNetB0) dan Fine-Tuning.\n",
    "3.  **Perbandingan:** Menganalisis dan membandingkan hasil dari ketiga pendekatan tersebut."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Langkah 1: Setup Lingkungan & Download Dataset\n",
    "\n",
    "Pertama, kita akan menginstal library Kaggle, mengautentikasi, dan mengunduh dataset yang diperlukan langsung ke lingkungan Colab."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"Installing Kaggle library...\")\n",
    "!pip install -q kaggle\n",
    "\n",
    "from google.colab import files\n",
    "import os\n",
    "\n",
    "print(\"\\nPlease upload your kaggle.json file\")\n",
    "files.upload() # Upload your kaggle.json API token here\n",
    "\n",
    "# Setup direktori Kaggle\n",
    "!mkdir -p ~/.kaggle\n",
    "!cp kaggle.json ~/.kaggle/\n",
    "!chmod 600 ~/.kaggle/kaggle.json\n",
    "\n",
    "print(\"\\nDownloading and unzipping the dataset...\")\n",
    "# Download dataset dari Kaggle\n",
    "!kaggle datasets download -d farukalam/tomato-leaf-diseases-detection-computer-vision -p /content/ --unzip\n",
    "\n",
    "print(\"\\nDataset ready.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Langkah 2: Import Library dan Definisikan Parameter"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import tensorflow as tf\n",
    "from tensorflow.keras.preprocessing.image import ImageDataGenerator\n",
    "from tensorflow.keras.models import Sequential, Model\n",
    "from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D\n",
    "from tensorflow.keras.applications import EfficientNetB0\n",
    "from tensorflow.keras.optimizers import Adam\n",
    "from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "\n",
    "# Definisikan Konstanta\n",
    "TRAIN_DIR = '/content/tomato/train'\n",
    "VAL_DIR = '/content/tomato/val'\n",
    "IMG_SIZE = (128, 128)\n",
    "BATCH_SIZE = 32\n",
    "NUM_CLASSES = 11"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Bagian 1: Reproduksi Model Baseline (Custom CNN)\n",
    "\n",
    "Di bagian ini, kita akan mereplikasi arsitektur dan proses training dari notebook Kaggle referensi."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Preparing Data for Baseline Model ---\")\n",
    "\n",
    "# Gunakan ImageDataGenerator seperti di notebook referensi\n",
    "train_datagen_baseline = ImageDataGenerator(\n",
    "    rescale=1./255,\n",
    "    shear_range=0.2,\n",
    "    zoom_range=0.2,\n",
    "    horizontal_flip=True\n",
    ")\n",
    "\n",
    "val_datagen_baseline = ImageDataGenerator(rescale=1./255)\n",
    "\n",
    "train_generator_baseline = train_datagen_baseline.flow_from_directory(\n",
    "    TRAIN_DIR,\n",
    "    target_size=IMG_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical'\n",
    ")\n",
    "\n",
    "validation_generator_baseline = val_datagen_baseline.flow_from_directory(\n",
    "    VAL_DIR,\n",
    "    target_size=IMG_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical',\n",
    "    shuffle=False\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Building Baseline CNN Model ---\")\n",
    "\n",
    "baseline_model = Sequential([\n",
    "    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),\n",
    "    MaxPooling2D(2, 2),\n",
    "    \n",
    "    Conv2D(64, (3, 3), activation='relu'),\n",
    "    MaxPooling2D(2, 2),\n",
    "    \n",
    "    Conv2D(128, (3, 3), activation='relu'),\n",
    "    MaxPooling2D(2, 2),\n",
    "    \n",
    "    Flatten(),\n",
    "    \n",
    "    Dense(128, activation='relu'),\n",
    "    Dense(NUM_CLASSES, activation='softmax')\n",
    "])\n",
    "\n",
    "baseline_model.compile(\n",
    "    optimizer='adam',\n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "baseline_model.summary()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Training Baseline CNN Model ---\")\n",
    "# Latih untuk epoch lebih sedikit agar cepat, notebook asli 50 epoch\n",
    "EPOCHS_BASELINE = 25 \n",
    "\n",
    "history_baseline = baseline_model.fit(\n",
    "    train_generator_baseline,\n",
    "    epochs=EPOCHS_BASELINE,\n",
    "    validation_data=validation_generator_baseline\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Evaluating Baseline Model ---\")\n",
    "results_baseline = baseline_model.evaluate(validation_generator_baseline)\n",
    "print(f\"Baseline Model Validation Loss: {results_baseline[0]:.4f}\")\n",
    "print(f\"Baseline Model Validation Accuracy: {results_baseline[1]*100:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Bagian 2: Eksplorasi dengan Transfer Learning & Fine-Tuning\n",
    "\n",
    "Sekarang kita akan menggunakan model pra-terlatih (EfficientNetB0) untuk mendapatkan performa yang lebih tinggi."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Preparing Data for Transfer Learning Model ---\")\n",
    "\n",
    "# Untuk transfer learning, kita hanya perlu normalisasi. Augmentasi akan dilakukan oleh lapisan Keras.\n",
    "train_datagen_tl = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)\n",
    "val_datagen_tl = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)\n",
    "\n",
    "train_generator_tl = train_datagen_tl.flow_from_directory(\n",
    "    TRAIN_DIR,\n",
    "    target_size=IMG_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical'\n",
    ")\n",
    "\n",
    "validation_generator_tl = val_datagen_tl.flow_from_directory(\n",
    "    VAL_DIR,\n",
    "    target_size=IMG_SIZE,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    class_mode='categorical',\n",
    "    shuffle=False\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Building Transfer Learning Model (EfficientNetB0) ---\")\n",
    "\n",
    "# 1. Muat Base Model\n",
    "base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))\n",
    "base_model.trainable = False # Bekukan bobot base model\n",
    "\n",
    "# 2. Tambahkan Lapisan Augmentasi dan Classifier baru\n",
    "data_augmentation = Sequential([\n",
    "    tf.keras.layers.RandomFlip('horizontal'),\n",
    "    tf.keras.layers.RandomRotation(0.2),\n",
    "    tf.keras.layers.RandomZoom(0.2),\n",
    "], name='data_augmentation')\n",
    "\n",
    "inputs = tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))\n",
    "x = data_augmentation(inputs)\n",
    "x = base_model(x, training=False) # Pastikan base model dalam inference mode\n",
    "x = GlobalAveragePooling2D()(x)\n",
    "x = Dropout(0.5)(x)\n",
    "outputs = Dense(NUM_CLASSES, activation='softmax')(x)\n",
    "\n",
    "tl_model = Model(inputs, outputs)\n",
    "\n",
    "tl_model.compile(\n",
    "    optimizer=Adam(learning_rate=1e-3),\n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "tl_model.summary()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Training Transfer Learning Model (Phase 1: Feature Extraction) ---\")\n",
    "EPOCHS_TL = 15\n",
    "\n",
    "history_tl = tl_model.fit(\n",
    "    train_generator_tl,\n",
    "    epochs=EPOCHS_TL,\n",
    "    validation_data=validation_generator_tl\n",
    ")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Fine-Tuning\n",
    "\n",
    "Setelah model konvergen pada lapisan baru, kita akan 'mencairkan' beberapa lapisan teratas dari base model dan melatihnya kembali dengan learning rate yang sangat kecil."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Fine-Tuning the Transfer Learning Model (Phase 2) ---\")\n",
    "\n",
    "# Cairkan base model\n",
    "base_model.trainable = True\n",
    "\n",
    "# Bekukan semua lapisan kecuali 20 terakhir\n",
    "for layer in base_model.layers[:-20]:\n",
    "    layer.trainable = False\n",
    "\n",
    "# Kompilasi ulang dengan learning rate yang sangat kecil\n",
    "tl_model.compile(\n",
    "    optimizer=Adam(learning_rate=1e-5), \n",
    "    loss='categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "tl_model.summary()\n",
    "\n",
    "# Lanjutkan pelatihan\n",
    "EPOCHS_FT = 10\n",
    "history_ft = tl_model.fit(\n",
    "    train_generator_tl,\n",
    "    epochs=EPOCHS_TL + EPOCHS_FT, # Lanjutkan dari epoch sebelumnya\n",
    "    initial_epoch=history_tl.epoch[-1] + 1,\n",
    "    validation_data=validation_generator_tl\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"--- Evaluating Fine-Tuned Model ---\")\n",
    "results_ft = tl_model.evaluate(validation_generator_tl)\n",
    "print(f\"Fine-Tuned Model Validation Loss: {results_ft[0]:.4f}\")\n",
    "print(f\"Fine-Tuned Model Validation Accuracy: {results_ft[1]*100:.2f}%\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Bagian 3: Kesimpulan dan Perbandingan Hasil"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"=========================================================\")\n",
    "print(\"               PERBANDINGAN AKHIR HASIL                  \")\n",
    "print(\"=========================================================\")\n",
    "print(f\"Model Baseline (Custom CNN): \\t{results_baseline[1]*100:.2f}% Akurasi Validasi\")\n",
    "print(f\"Model Fine-Tuned (EfficientNetB0): \\t{results_ft[1]*100:.2f}% Akurasi Validasi\")\n",
    "print(\"=========================================================\")\n",
    "\n",
    "# Menggabungkan history untuk plot\n",
    "acc = history_baseline.history['accuracy'] + history_tl.history['accuracy'] + history_ft.history['accuracy']\n",
    "val_acc = history_baseline.history['val_accuracy'] + history_tl.history['val_accuracy'] + history_ft.history['val_accuracy']\n",
    "loss = history_baseline.history['loss'] + history_tl.history['loss'] + history_ft.history['loss']\n",
    "val_loss = history_baseline.history['val_loss'] + history_tl.history['val_loss'] + history_ft.history['val_loss']\n",
    "\n",
    "plt.figure(figsize=(16, 6))\n",
    "\n",
    "# Plot Akurasi\n",
    "plt.subplot(1, 2, 1)\n",
    "plt.plot(history_baseline.history['val_accuracy'], label='Baseline Val Acc')\n",
    "plt.plot(history_tl.history['val_accuracy'] + history_ft.history['val_accuracy'], label='Transfer Learning Val Acc', color='orange')\n",
    "plt.axvline(EPOCHS_TL, color='grey', linestyle='--', label='Start Fine-Tuning')\n",
    "plt.title('Perbandingan Akurasi Validasi')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Akurasi')\n",
    "plt.legend()\n",
    "\n",
    "# Plot Loss\n",
    "plt.subplot(1, 2, 2)\n",
    "plt.plot(history_baseline.history['val_loss'], label='Baseline Val Loss')\n",
    "plt.plot(history_tl.history['val_loss'] + history_ft.history['val_loss'], label='Transfer Learning Val Loss', color='orange')\n",
    "plt.axvline(EPOCHS_TL, color='grey', linestyle='--', label='Start Fine-Tuning')\n",
    "plt.title('Perbandingan Loss Validasi')\n",
    "plt.xlabel('Epoch')\n",
    "plt.ylabel('Loss')\n",
    "plt.legend()\n",
    "\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

: 